In [ ]:
import os
os.environ["CUDA_DEVICE_ORDER"] = "PCI_BUS_ID"
os.environ["CUDA_VISIBLE_DEVICES"] = "3"
os.environ["KAGGLE_USERNAME"] = 'hyunw3'
os.environ["KAGGLE_KEY"] = "b48a329f42b53a6bda2afd4cf341bcf2"
# https://www.kaggle.com/dlaststark/tps-1021-lightgbm-with-twist
import tensorflow as tf
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import gc

In [2]:
import torch
import lightgbm as lgb
from sklearn.metrics import roc_auc_score, confusion_matrix, classification_report
from sklearn.model_selection import StratifiedKFold

In [3]:
torch.cuda.set_per_process_memory_fraction(0.3,0)

In [4]:
train_data = pd.read_csv("./train.csv")
train_data.set_index('id',inplace=True)
train_data.head()

,f0,f1,f2,f3,f4,f5,f6,f7,f8,f9,...,f276,f277,f278,f279,f280,f281,f282,f283,f284,target
id,,,,,,,,,,,,,,,,,,,,,
0,0.205979,0.410993,0.176775,0.223581,0.423543,0.476140,0.413590,0.612021,0.534873,0.147295,...,0,1,0,0,0,0,0,0,0,1
1,0.181004,0.473119,0.011734,0.213657,0.619678,0.441593,0.230407,0.686013,0.281971,0.238509,...,0,1,0,0,0,0,0,0,0,1
2,0.182583,0.307431,0.325950,0.207116,0.605699,0.309695,0.493337,0.751107,0.536272,0.286813,...,0,0,0,1,1,0,0,0,0,1
3,0.180240,0.494592,0.008367,0.223580,0.760618,0.439211,0.432055,0.776147,0.483958,0.260886,...,0,0,0,0,1,0,0,0,0,1
4,0.177172,0.495513,0.014263,0.548819,0.625396,0.562493,0.117158,0.561255,0.077115,0.158321,...,0,1,1,0,1,0,0,1,0,1


In [5]:
test_data = pd.read_csv("./test.csv")
test_data.set_index('id',inplace=True)
test_data.head()

,f0,f1,f2,f3,f4,f5,f6,f7,f8,f9,...,f275,f276,f277,f278,f279,f280,f281,f282,f283,f284
id,,,,,,,,,,,,,,,,,,,,,
1000000,0.178216,0.435617,0.010230,0.202074,0.390170,0.324221,0.221722,0.738894,0.582588,0.343770,...,1,0,0,0,0,0,1,1,1,0
1000001,0.181250,0.476455,0.022413,0.283146,0.598020,0.349508,0.283467,0.721575,0.268990,0.208373,...,0,0,0,0,0,0,0,0,0,0
1000002,0.159721,0.451202,0.259649,0.365274,0.594634,0.413502,0.249318,0.642339,0.411104,0.246891,...,0,0,0,0,0,0,1,0,0,0
1000003,0.182424,0.520976,0.095344,0.327742,0.741830,0.358711,0.270077,0.601662,0.297742,0.252829,...,0,0,0,0,0,1,1,0,0,0
1000004,0.229329,0.336513,0.023511,0.300913,0.668738,0.481586,0.545660,0.667849,0.546045,0.202731,...,0,0,0,0,1,0,0,1,0,0


In [6]:
label='target'
print("summary of class variable : \n",train_data[label].describe())

summary of class variable : 
 count    1000000.000000
mean           0.500485
std            0.500000
min            0.000000
25%            0.000000
50%            1.000000
75%            1.000000
max            1.000000
Name: target, dtype: float64


In [7]:
FOLD = 8
SEEDS = [17, 29, 37]

In [8]:
trainX = train_data.drop(['target'],axis=1)
trainY = train_data['target']
trainX.shape, trainY.shape

((1000000, 285), (1000000,))

In [9]:
params = {
    'objective' : 'binary',
    'metric' : 'auc',
    'colsample_bytree' : 0.4,
    'subsample' : 0.6,
    'subsample_freq' : 1, 
    'reg_alpha' : 10,
    'reg_lambda' : 0.1,
    'min_child_weight' : 256,
    'min_child_samples' : 20,
    'verbosity' : 0,
    'force_col_wise' : True,
    'random_state' : 2021
}

In [10]:
counter = 0
oof_score = 0
y_pred_final_lgb = np.zeros((test_data.shape[0], 1))
y_pred_meta_lgb = np.zeros((train_data.shape[0], 1))

for sidx, seed in enumerate(SEEDS):
    seed_score =0
    kfold = StratifiedKFold(n_splits=FOLD, shuffle=True, random_state = seed)
    for idx, (train,val) in enumerate(kfold.split(trainX,trainY)):
        counter+=1
        train_x, train_y = trainX.iloc[train], trainY.iloc[train]
        val_x, val_y = trainX.iloc[val], trainY.iloc[val]
        
        lgbtrain = lgb.Dataset(train_x, label=train_y.ravel(), free_raw_data=False)
        lgbval = lgb.Dataset(val_x, label=val_y.ravel(), free_raw_data=False)
        params['learning_rate'] = 0.05
        
        model = lgb.train(params, lgbtrain, valid_sets=[lgbtrain, lgbval],
                          early_stopping_rounds=200,verbose_eval=500)
        params['learning_rate'] = 0.01
        model = lgb.train(params, lgbtrain, valid_sets=[lgbtrain, lgbval],
                         init_model=model,early_stopping_rounds=200,verbose_eval=500)
        
        y_pred = model.predict(val_x, num_iteration=model.best_iteration)
        
        y_pred_meta_lgb[val] += np.array([y_pred]).T
        y_pred_final_lgb += np.array([model.predict(test_data, num_iteration=model.best_iteration)]).T
        score = roc_auc_score(val_y, y_pred)
        
        oof_score += score
        seed_score += score
        
        print("\nLightGBM | Seed-{} | Fold-{} | OOF Score: {}\n".format(seed, idx, score))
        
        del model, y_pred
        del train_x, train_y
        del val_x, val_y
        gc.collect()
    print("\nLightGBM | Seed: {} | Aggregate OOF Score: {}\n\n".format(seed, (seed_score / FOLD)))
y_pred_meta_lgb = y_pred_meta_lgb / float(len(SEEDS))
y_pred_final_lgb = y_pred_final_lgb / float(counter)
oof_score /= float(counter)
print("LightGBM | Aggregate OOF Score: {}".format(oof_score))

Training until validation scores don't improve for 200 rounds
Did not meet early stopping. Best iteration is:
[98]	training's auc: 0.846241	valid_1's auc: 0.846007
Training until validation scores don't improve for 200 rounds
Did not meet early stopping. Best iteration is:
[196]	training's auc: 0.847616	valid_1's auc: 0.847162

LightGBM | Seed-17 | Fold-0 | OOF Score: 0.8471619728101634

Training until validation scores don't improve for 200 rounds
Did not meet early stopping. Best iteration is:
[98]	training's auc: 0.846427	valid_1's auc: 0.844955
Training until validation scores don't improve for 200 rounds
Did not meet early stopping. Best iteration is:
[196]	training's auc: 0.847797	valid_1's auc: 0.846092

LightGBM | Seed-17 | Fold-1 | OOF Score: 0.8460922041111274

Training until validation scores don't improve for 200 rounds
Did not meet early stopping. Best iteration is:
[98]	training's auc: 0.846488	valid_1's auc: 0.843999
Training until validation scores don't improve for 200

In [11]:
np.savez_compressed('./TPS_1021_LGB_Meta_Features.npz',
                    y_pred_meta_lgb=y_pred_meta_lgb,
                    y_pred_final_lgb=y_pred_final_lgb)

In [12]:
submit_df = pd.read_csv("./sample_submission.csv")
submit_df['target'] = y_pred_final_lgb.ravel()
submit_df.to_csv("LGB_Submission.csv", index=False)
submit_df.head()

,id,target
0,1000000,0.700541
1,1000001,0.305274
2,1000002,0.817841
3,1000003,0.764720
4,1000004,0.249729


In [15]:
submit_df['target'] >=0.5

0          True
1         False
2          True
3          True
4         False
          ...  
499995     True
499996     True
499997    False
499998     True
499999    False
Name: target, Length: 500000, dtype: bool